In [16]:
%%writefile practice74.cu
// Practice 7 Доп. задание 2
// 2.  Реализуйте алгоритм Blelloch Scan для более эффективного сканирования.

#include <iostream>               // Для стандартного ввода-вывода
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL: Blelloch
__global__ void blockScanKernel(int *d_input, int *d_output, int *d_sums, int n) {
    extern __shared__ int temp[];          // Создаем shared memory для одного блока
    int tid = threadIdx.x;                  // Локальный индекс потока в блоке
    int i = blockIdx.x * blockDim.x + tid; // Глобальный индекс элемента массива

    // Загружаем элементы массива в shared memory
    temp[tid] = (i < n) ? d_input[i] : 0;  // Если индекс вне массива, кладем 0
    __syncthreads();                        // Синхронизация потоков блока

    // UPSWEEP (шаг вверх)
    for (int offset = 1; offset < blockDim.x; offset *= 2) { // Проходим по уровням дерева сумм
        int idx = (tid + 1) * offset * 2 - 1;                // Индекс элемента для суммирования
        if (idx < blockDim.x)
            temp[idx] += temp[idx - offset];                 // Складываем пары элементов
        __syncthreads();                                    // Синхронизация после каждого шага
    }

    // Сохраняем сумму блока в массив сумм блоков для рекурсивного скана
    if (tid == 0 && d_sums != nullptr)
        d_sums[blockIdx.x] = temp[blockDim.x - 1];         // Последний элемент — сумма всего блока

    // DOWNSWEEP (шаг вниз)
    if (tid == 0)
        temp[blockDim.x - 1] = 0;                          // Обнуляем последний элемент перед downsweep
    __syncthreads();                                      // Синхронизация потоков

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) { // Проходим обратно по дереву
        int idx = (tid + 1) * offset * 2 - 1;                     // Индекс для обновления
        if (idx < blockDim.x) {
            int t = temp[idx - offset];                 // Временная переменная
            temp[idx - offset] = temp[idx];            // Перемещаем значение
            temp[idx] += t;                             // Добавляем сохраненное значение
        }
        __syncthreads();                                // Синхронизация после каждого шага
    }

    // Копируем результаты обратно в глобальную память
    if (i < n)
        d_output[i] = temp[tid];                        // Записываем элемент блока в результат
}

// CUDA KERNEL: СУММИРОВАНИЕ ПРЕФИКСНЫХ БЛОКОВ
__global__ void addBlockSums(int *d_output, int *d_sums, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;       // Глобальный индекс элемента
    if (blockIdx.x > 0 && i < n)                         // Пропускаем первый блок
        d_output[i] += d_sums[blockIdx.x - 1];          // Добавляем сумму предыдущего блока
}

// ФУНКЦИЯ ДЛЯ BLELLOCH SCAN
void blellochScan(int *d_input, int *d_output, int N) {
    int threadsPerBlock = 1024;                                           // Потоки на блок
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;       // Количество блоков

    int *d_block_sums = nullptr;                                          // Указатель для массива сумм блоков

    if (blocksPerGrid > 1)                                                // Если блоков больше одного
        cudaMalloc(&d_block_sums, blocksPerGrid * sizeof(int));           // Выделяем память под суммы блоков

    // Шаг 1: Сканируем каждый блок
    blockScanKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, d_block_sums, N);

    // Шаг 2: Если блоков > 1, рекурсивно сканируем массив сумм блоков
    if (blocksPerGrid > 1) {
        int *d_sums_scan;                                                 // Временный массив для рекурсивного скана
        cudaMalloc(&d_sums_scan, blocksPerGrid * sizeof(int));            // Выделяем память
        blellochScan(d_block_sums, d_sums_scan, blocksPerGrid);           // Рекурсивный вызов функции
        addBlockSums<<<blocksPerGrid, threadsPerBlock>>>(d_output, d_sums_scan, N); // Добавляем суммы блоков
        cudaFree(d_sums_scan);                                            // Освобождаем временный массив
    }

    if (blocksPerGrid > 1)
        cudaFree(d_block_sums);                                           // Освобождаем массив сумм блоков
}

// ФУНКЦИЯ ТЕСТИРОВАНИЯ
void runBlellochTest(int N) {
    cout << "\nРазмер массива: " << N << endl;

    int *h_array = new int[N];                  // Массив на CPU
    for (int i = 0; i < N; i++)
        h_array[i] = 1;                         // Заполняем массив единицами

    // CPU SCAN
    int *h_scan_cpu = new int[N];               // Массив для CPU результата
    auto cpu_start = chrono::high_resolution_clock::now(); // Старт таймера CPU
    h_scan_cpu[0] = h_array[0];                // Первый элемент скана
    for (int i = 1; i < N; i++)
        h_scan_cpu[i] = h_scan_cpu[i - 1] + h_array[i]; // Сканируем на CPU
    auto cpu_end = chrono::high_resolution_clock::now();  // Остановка таймера CPU
    chrono::duration<double> cpu_time = cpu_end - cpu_start; // Вычисление времени CPU

    // GPU SCAN
    int *d_input, *d_output;                    // Указатели на GPU память
    cudaMalloc(&d_input, N * sizeof(int));      // Выделяем память под вход
    cudaMalloc(&d_output, N * sizeof(int));     // Выделяем память под выход
    cudaMemcpy(d_input, h_array, N * sizeof(int), cudaMemcpyHostToDevice); // Копируем данные CPU -> GPU

    cudaEvent_t gpu_start, gpu_stop;            // CUDA события для таймера GPU
    cudaEventCreate(&gpu_start);                // Создаем событие старта
    cudaEventCreate(&gpu_stop);                 // Создаем событие окончания
    cudaEventRecord(gpu_start);                 // Запускаем таймер GPU

    blellochScan(d_input, d_output, N);         // Запускаем рекурсивный Blelloch Scan

    cudaEventRecord(gpu_stop);                  // Останавливаем таймер GPU
    cudaEventSynchronize(gpu_stop);             // Ждем окончания события
    float gpu_time = 0;
    cudaEventElapsedTime(&gpu_time, gpu_start, gpu_stop); // Получаем время GPU

    // Вывод времени
    cout << "CPU время: " << cpu_time.count() * 1000 << " мс" << endl;
    cout << "GPU время: " << gpu_time << " мс" << endl;

    // Освобождение памяти
    delete[] h_array;                            // Освобождаем массив CPU
    delete[] h_scan_cpu;                         // Освобождаем CPU результат
    cudaFree(d_input);                           // Освобождаем GPU память
    cudaFree(d_output);
}
// Основная функция
int main() {
    runBlellochTest(1024);        // Тест массива
    runBlellochTest(1000000);     // Тест массива
    runBlellochTest(10000000);    // Тест массива
    return 0;                      // Завершение программы
}


Overwriting practice74.cu


In [17]:
# Компиляция
!nvcc practice74.cu -o practice74 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                    # -std=c++11 — стандарт C++
# Запуск
!./practice74


Размер массива: 1024
CPU время: 0.005538 мс
GPU время: 0.12496 мс

Размер массива: 1000000
CPU время: 4.70519 мс
GPU время: 0.646688 мс

Размер массива: 10000000
CPU время: 46.9139 мс
GPU время: 4.89974 мс
